# Machine Learning (ML): Google Cloud Platform (GCP): TensorFlow (TF): Financial Time-Series
# Model 01 = Binary Classifier
# Model 02 = Neural Network

## 1) Import Libraries

In [1]:
import StringIO

import pandas as pd
from pandas.tools.plotting import autocorrelation_plot
from pandas.tools.plotting import scatter_matrix

import numpy as np

import matplotlib.pyplot as plt

import gcp
import gcp.bigquery as bq

import tensorflow as tf

from IPython.core.display import HTML


## 2) Install stuff

In [2]:
# Install xlrd - required for reading excel files
# !pip install xlrd
!pip install xlrd
# -----------------------------------------------------------------------------------------------
# Downloading/unpacking xlrd
#   Downloading xlrd-1.0.0.tar.gz (2.6MB): 2.6MB downloaded
#   Running setup.py (path:/tmp/pip-build-brL6Op/xlrd/setup.py) egg_info for package xlrd
    
#     warning: no files found matching 'README.html'
# Installing collected packages: xlrd
#   Running setup.py install for xlrd
#     changing mode of build/scripts-2.7/runxlrd.py from 644 to 755
    
#     warning: no files found matching 'README.html'
#     changing mode of /usr/local/bin/runxlrd.py to 755
#   Could not find .egg-info directory in install record for xlrd
# Successfully installed xlrd
# Cleaning up...
# -----------------------------------------------------------------------------------------------


Traceback (most recent call last):
  File "/usr/local/bin/pip", line 9, in <module>
    load_entry_point('pip==1.5.6', 'console_scripts', 'pip')()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 542, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2569, in load_entry_point
    return ep.load()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2229, in load
    return self.resolve()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2235, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/usr/lib/python2.7/dist-packages/pip/__init__.py", line 74, in <module>
    from pip.vcs import git, mercurial, subversion, bazaar  # noqa
  File "/usr/lib/python2.7/dist-packages/pip/vcs/mercurial.py", line 9, in <module>
    from pip.download import path_t

In [3]:
# Install dropbox - required for writing files to dropbox, not required for reading files from dropbox
!pip install dropbox
# -----------------------------------------------------------------------------------------------
# Downloading/unpacking dropbox
#   Downloading dropbox-6.6.2-py2-none-any.whl (261kB): 261kB downloaded
# Requirement already satisfied (use --upgrade to upgrade): six>=1.3.0 in /usr/local/lib/python2.7/dist-packages (from dropbox)
# Requirement already satisfied (use --upgrade to upgrade): urllib3 in /usr/lib/python2.7/dist-packages (from dropbox)
# Downloading/unpacking requests!=2.6.1,>=2.5.1 (from dropbox)
#   Downloading requests-2.11.0-py2.py3-none-any.whl (514kB): 514kB downloaded
# Downloading/unpacking typing>=3.5.2 (from dropbox)
#   Downloading typing-3.5.2.2.tar.gz (51kB): 51kB downloaded
#   Running setup.py (path:/tmp/pip-build-xSBpJ2/typing/setup.py) egg_info for package typing
    
# Installing collected packages: dropbox, requests, typing
#   Found existing installation: requests 2.4.3
#     Not uninstalling requests at /usr/lib/python2.7/dist-packages, owned by OS
#   Running setup.py install for typing
    
#   Could not find .egg-info directory in install record for typing>=3.5.2 (from dropbox)
# Successfully installed dropbox requests typing
# Cleaning up...
# -----------------------------------------------------------------------------------------------


Traceback (most recent call last):
  File "/usr/local/bin/pip", line 9, in <module>
    load_entry_point('pip==1.5.6', 'console_scripts', 'pip')()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 542, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2569, in load_entry_point
    return ep.load()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2229, in load
    return self.resolve()
  File "/usr/local/lib/python2.7/dist-packages/pkg_resources/__init__.py", line 2235, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/usr/lib/python2.7/dist-packages/pip/__init__.py", line 74, in <module>
    from pip.vcs import git, mercurial, subversion, bazaar  # noqa
  File "/usr/lib/python2.7/dist-packages/pip/vcs/mercurial.py", line 9, in <module>
    from pip.download import path_t

## 3) Get the data from dropbox excel file

In [18]:
# Save file on dropbox.com

fx_mxn_url_data_raw = 'https://www.dropbox.com/s/foso5ckzl9vrui0/fx_mxn_data_raw.xlsm'

# force = 1 # will grab from dropbox to gcp
force = 0 # will use if already in gcp


In [19]:
# Import file using url to file on dropbox
from urlparse import urlparse
from os.path import basename

fx_mxn_bn_data_raw = basename(urlparse(fx_mxn_url_data_raw).path)
print 'fx_mxn_bn_data_raw=' + fx_mxn_bn_data_raw
print 'already_downloaded=' + str(already_downloaded)

try:
    already_downloaded
except:
    already_downloaded = False
    
if force or not already_downloaded:
    already_downloaded = True
    !rm $fx_mxn_bn_data_raw
    !wget $fx_mxn_bn_data_raw
#!ls -l

fx_mxn_bn_data_raw=fx_mxn_data_raw.xlsm
already_downloaded=True


## 4) Read the Excel file raw data

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xlrd

# Read the excel data
fx_mxn_df_data_raw = pd.read_excel(fx_mxn_bn_data_raw, 'data_raw')
# data = pd.read_excel(bn_data_raw, 'data_raw', header=0, index_col=0, parse_cols=None)
  # parse_cols=None: parse all columns
  # header=0:    sets the row 0 as col labels (ie. headers)
  # index_col=0: sets the col 0 as row labels (ie. index)
fx_mxn_df_data_raw.head() # display the first few lines

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,Ticker,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty
1,Date,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty
2,2016-05-27 00:00:00,18.4751,289.95,55.4,141.28,424.14,111.85,118.813,66.0342,35645.2,...,2.102,2.138,-10.63,-18,-31.63,1.943,2.074,2.133,2.1738,124.13
3,2016-05-26 00:00:00,18.4544,284.68,55.29,141.83,427.999,111.845,118.75,65.282,35316.9,...,2.104,2.139,-10.46,-17.65,-31.43,1.928,2.0575,2.125,2.1598,124.3
4,2016-05-25 00:00:00,18.4852,281.4,56.33,140.98,5392.86,111.825,118.438,65.9242,35680.2,...,2.107,2.14,-11.11,-18.4,-32.03,1.955,2.0825,2.1453,2.1785,124.11


In [21]:
print type(fx_mxn_df_data_raw)
# <class 'pandas.core.frame.DataFrame'>

<class 'pandas.core.frame.DataFrame'>


## 4.1) Write the Excel file back to dropbox

In [22]:
# Include the Dropbox SDK
import dropbox

## 4.1.1) Method 1

In [23]:
# # Get your app key and secret from the Dropbox developer website
# app_key = '9zgzo4d4s8tco7c'
# app_secret = '5iq9409uedmichl'

# flow = dropbox.client.DropboxOAuth2FlowNoRedirect(app_key, app_secret)

In [24]:
# authorize_url = flow.start()

In [25]:
# print '1. Go to: ' + authorize_url
# print '2. Click "Allow" (you might have to log in first)'
# print '3. Copy the authorization code.'
# code = raw_input("Enter the authorization code here: ").strip()

# # output
# # 1. Go to: https://www.dropbox.com/1/oauth2/authorize?response_type=code&client_id=9zgzo4d4s8tco7c
# # 2. Click "Allow" (you might have to log in first)
# # 3. Copy the authorization code.
# # Enter the authorization code here: Z0hHmWIPD4AAAAAAAAAAE2kOeZ0UPKiQi9eeQFEyY1s
# # Enter the authorization code here: Z0hHmWIPD4AAAAAAAAAAFTmf6b0e8U4eOd96tCcbILs

In [26]:
# This will fail if the user enters an invalid authorization code
# access_token, user_id = flow.finish(code)


In [27]:
# client = dropbox.client.DropboxClient(access_token)
# print 'linked account: ', client.account_info()

# # linked account:  {u'referral_link': u'https://db.tt/wTxYqge4', u'display_name': u'Eyup Saltik', u'uid': 576395988, 
# # u'locale': u'en', u'email_verified': True, u'email': u'eyup.saltik.2016@gmail.com', u'is_paired': False, 
# # u'team': None, u'name_details': {u'familiar_name': u'Eyup', u'surname': u'Saltik', u'given_name': u'Eyup'}, 
# # u'country': u'SG', u'quota_info': {u'datastores': 0, u'shared': 0, u'quota': 2147483648, u'normal': 40119203}}
  

In [28]:
# response = client.put_file('/working-final.csv', str(data_raw))
# print "uploaded:", response

## 4.1.2) Method 2

In [29]:
s_dropbox_access_token = 'Z0hHmWIPD4AAAAAAAAAADKBYofFVGTUnVcva8HKUpKCj6SDaY15WNHIZcRcEknoO'
client = dropbox.client.DropboxClient(s_dropbox_access_token)
print 'linked account: ', client.account_info()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  from ipykernel import kernelapp as app


linked account:  {u'referral_link': u'https://db.tt/wTxYqge4', u'display_name': u'Eyup Saltik', u'uid': 576395988, u'locale': u'en', u'email_verified': True, u'email': u'eyup.saltik.2016@gmail.com', u'is_paired': False, u'team': None, u'name_details': {u'familiar_name': u'Eyup', u'surname': u'Saltik', u'given_name': u'Eyup'}, u'country': u'SG', u'quota_info': {u'datastores': 0, u'shared': 0, u'quota': 2147483648, u'normal': 7193548}}


In [30]:
response = client.put_file('/fx_mxn_data_raw_from_GCP.csv', str(fx_mxn_df_data_raw.to_csv()), overwrite=True)
print "uploaded:", response

uploaded: {u'icon': u'page_white_excel', u'bytes': 2672933, u'thumb_exists': False, u'rev': u'424a1e9d47', u'modified': u'Tue, 23 Aug 2016 06:52:27 +0000', u'shareable': False, u'client_mtime': u'Tue, 23 Aug 2016 06:52:27 +0000', u'path': u'/fx_mxn_data_raw_from_GCP.csv', u'is_dir': False, u'size': u'2.5 MB', u'root': u'dropbox', u'mime_type': u'text/csv', u'revision': 66}


## 5) Select data df

In [37]:
df = fx_mxn_df_data_raw
df.head()

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,Ticker,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty
1,Date,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty
2,2016-05-27 00:00:00,18.4751,289.95,55.4,141.28,424.14,111.85,118.813,66.0342,35645.2,...,2.102,2.138,-10.63,-18,-31.63,1.943,2.074,2.133,2.1738,124.13
3,2016-05-26 00:00:00,18.4544,284.68,55.29,141.83,427.999,111.845,118.75,65.282,35316.9,...,2.104,2.139,-10.46,-17.65,-31.43,1.928,2.0575,2.125,2.1598,124.3
4,2016-05-25 00:00:00,18.4852,281.4,56.33,140.98,5392.86,111.825,118.438,65.9242,35680.2,...,2.107,2.14,-11.11,-18.4,-32.03,1.955,2.0825,2.1453,2.1785,124.11


## 6) df_ticker_desc = Just a table to map column id to the ticker and name of the variable

In [38]:
df_ticker_desc = df.ix[:1] # 1st 2 rows
df_ticker_desc

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,Ticker,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty
1,Date,USDMXN Curncy,ASURB MM Equity,BIMBOA MM Equity,CN1 Comdty,CT350188 Curncy,DU1 Comdty,ED749713@BVAL Corp,EMRUEMRU Index,EURMXN3Y Curncy,...,USSA25 ICPL Curncy,USSA30 ICPL Curncy,USSP15 CMPN Curncy,USSP20 CMPN Curncy,USSP25 CMPN Curncy,USSW15 CMPN Curncy,USSW20 CMPN Curncy,USSW25 CMPN Curncy,USSW30 CMPN Curncy,XQ1 Comdty


## 7) df_values = the actual values of the 86 variables

In [39]:
df_values = fx_mxn_df_data_raw_pca.ix[2:] # From row 2 onwards
df_values.head()

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
2,2016-05-27 00:00:00,18.4751,289.95,55.4,141.28,424.14,111.85,118.813,66.0342,35645.2,...,2.102,2.138,-10.63,-18,-31.63,1.943,2.074,2.133,2.1738,124.13
3,2016-05-26 00:00:00,18.4544,284.68,55.29,141.83,427.999,111.845,118.75,65.282,35316.9,...,2.104,2.139,-10.46,-17.65,-31.43,1.928,2.0575,2.125,2.1598,124.3
4,2016-05-25 00:00:00,18.4852,281.4,56.33,140.98,5392.86,111.825,118.438,65.9242,35680.2,...,2.107,2.14,-11.11,-18.4,-32.03,1.955,2.0825,2.1453,2.1785,124.11
5,2016-05-24 00:00:00,18.487,282.81,56.06,141.16,5392.86,111.815,118.5,66.8658,36154.1,...,2.126,2.158,-11.18,-18.43,-31.94,1.9469,2.0703,2.1305,2.1615,124.24
6,2016-05-23 00:00:00,18.5161,282.02,55.27,141.31,5392.86,111.815,120.188,67.0104,36684.3,...,2.092,2.125,-10.81,-18.14,-31.55,1.9245,2.0503,2.112,2.1435,124.27


## 8) Set the dates as index of the dataframe

In [ ]:
data_values_indexed = data_values.set_index([0])
data_values_indexed.head()

In [ ]:
data_values_indexed.tail()

## 9) Fill any gaps in the data

In [ ]:
# Pandas includes a very convenient function for filling gaps in the data.
data_values_indexed = data_values_indexed.fillna(method='ffill')
data_values_indexed.head()

# Exploratory data analysis

Exploratory Data Analysis (EDA) is foundational to working with machine learning, and any other sort of analysis. EDA means getting to know your data, getting your fingers dirty with your data, feeling it and seeing it. The end result is you know your data very well, so when you build models you build them based on an actual, practical, physical understanding of the data, not assumptions or vaguely held notions. You can still make assumptions of course, but EDA means you will understand your assumptions and why you're making those assumptions. 

First, take a look at the data.

## 10) Describe the data briefly

In [ ]:
data_values_indexed.describe()

You can see that the various indices operate on scales differing by orders of magnitude. It's best to scale the data so that, for example, operations involving multiple indices aren't unduly influenced by a single, massive index.


## 11) Plot the data

In [ ]:
# N.B. A super-useful trick-ette is to assign the return value of plot to _ 
# so that you don't get text printed before the plot itself.

# _ = pd.concat([data_values_indexed[1],
#   data_values_indexed[1070],
#   data_values_indexed[788],
#   data_values_indexed[926]], axis=1).plot(figsize=(20, 15))

_ = data_values_indexed.plot(figsize=(20, 15))

As expected, the structure isn't uniformly visible for the indices. Divide each value in an individual index by the maximum value for that index., and then replot. The maximum value of all indices will be 1.

## 12) Calculate the max value for each column, prepare to scale data for each column

In [ ]:
data_values_indexed_max = data_values_indexed.max(axis=0) # max across axis 0 = rows
data_values_indexed_max.head()

## 13) Scale the data

In [ ]:
data_values_indexed_scaled = data_values_indexed / data_values_indexed_max
data_values_indexed_scaled.head()

## 14) Plot the scaled data

In [ ]:
_ = data_values_indexed_scaled.plot(figsize=(20, 15))

## 15) Auto-correlations

Next, plot autocorrelations for each of the indices. The autocorrelations determine correlations between current values of the index and lagged values of the same index. The goal is to determine whether the lagged values are reliable indicators of the current values. If they are, then we've identified a correlation.

In [ ]:
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(10)

_ = autocorrelation_plot(data_values_indexed_scaled)

For 2300 lagged days, we observe positive auto-correlations.
This suggests that as the variables increase, they tend to keep on increasing. Momentum.

After 2300 lagged days, we observe negative auto-correlations.

## 16) Skip: Just a reminder of the PCA columns we selected earlier

In [ ]:
# list_cols_pca = [ 0,1,
#                   1070,788,926,112,69,574,654,1160,527,323,
#                   397,118,774,1028,1034,655,907,736,251,388,
#                   327,243,705,303,1146,467,136,1006,600,15,
#                   231,290,131,782,20,1048,630,1173,431,856,
#                   67,299,838,639,53,932,870,938,1061
#                 ]

## 17) Scatter plots of the first 20 of our 86 variables vs USDMXN(varid=1)

In [ ]:
# _ = scatter_matrix(data_values_indexed_scaled)
dvis = data_values_indexed_scaled

_ = scatter_matrix(
      pd.concat(
      [ 
        dvis[ 1],dvis[ 2],dvis[ 3],dvis[ 4],dvis[ 5],dvis[ 6],dvis[ 7],dvis[ 8],dvis[ 9],dvis[10],
        dvis[11],dvis[12],dvis[13],dvis[14],dvis[15],dvis[16],dvis[17],dvis[18],dvis[19],dvis[20],
      ], axis=1), figsize=(15, 15), diagonal='kde')

## 18) Remind ourselves what our scaled data (of price or index LEVELS) looks like

In [ ]:
data_values_indexed_scaled.head()

## 19) Calculate Log Returns on our scaled data

In [ ]:
data_values_indexed_scaled_logret = pd.DataFrame()
data_values_indexed_scaled_logret = np.log(data_values_indexed_scaled/data_values_indexed_scaled.shift(-1)) # note dates are reverse-chrono

data_values_indexed_scaled_logret.head()

In [ ]:
data_values_indexed_scaled_logret.tail()

In [ ]:
data_values_indexed_scaled_logret.describe()

In [ ]:
# 2000/01/07 to 2016/05/27 = 1 header row + 4279 data rows
# print data_values_indexed_scaled_logret.head()
# print data_values_indexed_scaled_logret.tail()
print 'NumRowsIncludeHeader = len(data_values_indexed_scaled_logret) = ' + str(len(data_values_indexed_scaled_logret))

## 20) Replace inf, NaN in data with Zero

In [ ]:
data_values_indexed_scaled_logret = data_values_indexed_scaled_logret.replace([np.inf, -np.inf, np.nan], 0)
data_values_indexed_scaled_logret.describe()

## 21) Skip: Fill the Gaps

In [ ]:
# Pandas includes a very convenient function for filling gaps in the data.
# data_values_indexed_scaled_logret = data_values_indexed_scaled_logret.fillna(method='ffill')
data_values_indexed_scaled_logret.head()

## 22) Plot log returns of scaled data

In [ ]:
_ = data_values_indexed_scaled_logret.plot(figsize=(20, 15))

## 23) Auto-Correlations of log returns of scaled data

In [ ]:
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(10)

_ = autocorrelation_plot(data_values_indexed_scaled_logret)

There are no auto-correlations, so we are good.

## 24) Skip: Just a reminder of the PCA50 Columns we selected earlier

In [ ]:
# list_cols_pca = [ 0,1,
#                   1070,788,926,112,69,574,654,1160,527,323,
#                   397,118,774,1028,1034,655,907,736,251,388,
#                   327,243,705,303,1146,467,136,1006,600,15,
#                   231,290,131,782,20,1048,630,1173,431,856,
#                   67,299,838,639,53,932,870,938,1061
#                 ]

## 25) Scatter plots of log returns of first 20 of the 86 variables where log-returns of USDMXN(varid=1)

In [ ]:
# _ = scatter_matrix(data_values_indexed_scaled_logret)   # takes long time, becareful, save work first
dvislr = data_values_indexed_scaled_logret

_ = scatter_matrix(
      pd.concat(
      [ 
        dvislr[ 1],dvislr[ 2],dvislr[ 3],dvislr[ 4],dvislr[ 5],dvislr[ 6],dvislr[ 7],dvislr[ 8],dvislr[ 9],dvislr[10],
        dvislr[11],dvislr[12],dvislr[13],dvislr[14],dvislr[15],dvislr[16],dvislr[17],dvislr[18],dvislr[19],dvislr[20],
      ], axis=1), figsize=(15, 15), diagonal='kde')


In [ ]:
data_values_indexed_scaled_logret.head()

## Summing up the EDA

At this point, you've done a good enough job of exploratory data analysis. You've visualized our data and come to know it better. 
You've transformed it into a form that is useful for modelling, log returns, and looked at how indices relate to each other. 

What should we think so far?

Cloud Datalab is working great. With just a few lines of code, you were able to munge the data, visualize the changes, and make decisions. You could easily analyze and iterate. This is a common feature of iPython, but the advantage here is that Cloud Datalab is a managed service that you can simply click and use, so you can focus on your analysis.

## Feature selection

At this point, we can see a model:

* We'll predict whether the USDMXN close today will be higher or lower than yesterday.

Predicting whether the log return of the USDMXN is positive or negative is a classification problem. 
That is, we want to choose one option from a finite set of options, in this case positive or negative. 
This is the base case of classification where we have only two values to choose from, known as binary classification, or logistic regression.

Machine learning models are very good at finding weak signals from data.
In machine learning, as in most things, there are subtle tradeoffs happening, but in general good data is better than good algorithms, which are better than good frameworks. 
You need all three pillars but in that order of importance: data, algorithms, frameworks.

## TensorFlow

[TensorFlow](https://tensorflow.org) is an open source software library, initiated by Google, for numerical computation using data flow graphs. TensorFlow is based on Google's machine learning expertise and is the next generation framework used internally at Google for tasks such as translation and image recognition. It's a wonderful framework for machine learning because it's expressive, efficient, and easy to use.

### Feature engineering for TensorFlow

From a training and testing perspective, time series data is easy. Training data should come from events that happened before test data events, and be contiguous in time.  Otherwise,  your model would be trained on events from "the future", at least as compared to the test data. It would then likely perform badly in practice, because you can’t really have access to data from the future. That means random sampling or cross validation don't apply to time series data. Decide on a training-versus-testing split, and divide your data into training and test datasets.

In this case, you'll create the features together with two additional columns:

* usdmxn_logret_positive, which is 1 if the log return of the USDMXN close is positive, and 0 otherwise. 
* usdmxn_logret_negative, which is 1 if the log return of the USDMXN close is negative, and 1 otherwise. 

We'll use 80% of our data for training and 20% for testing.

# Model 1: Binary Classification (BC) = Logistic Regression (LR)

## Model 1: Step 01: Indicator Columns

In [ ]:
# 00: Original: Initialize indicator columns to 0
data_values_indexed_scaled_logret['usdmxn_logret_positive'] = 0
data_values_indexed_scaled_logret['usdmxn_logret_negative'] = 0

In [ ]:
# 00: Populate results columns according to actual usdmxn returns (positive or negative)
data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] >= 0, 'usdmxn_logret_positive'] = 1
data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] <  0, 'usdmxn_logret_negative'] = 1

In [ ]:
# 01: Noise Cancellation
data_values_indexed_scaled_logret['usdmxn_logret_positive_0050bp'] = 0
data_values_indexed_scaled_logret['usdmxn_logret_negative_0050bp'] = 0

In [ ]:
# 01: Populate results columns according to actual usdmxn returns (outside noise zone above or below)
data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] >= 0.0050, 'usdmxn_logret_positive_0050bp'] = 1
data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] <  -0.0050, 'usdmxn_logret_negative_0050bp'] = 1

In [ ]:
# # 02: Stratification
# data_values_indexed_scaled_logret['usdmxn_logret_+0150bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0150bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0140bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0140bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0130bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0130bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0120bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0120bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0110bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0110bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0100bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0100bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0090bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0090bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0080bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0080bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0070bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0070bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0060bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0060bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0050bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0050bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0040bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0040bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0030bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0030bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0020bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0020bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0010bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0010bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_+0000bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_+0000bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0010bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0010bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0020bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0020bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0030bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0030bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0040bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0040bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0050bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0050bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0060bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0060bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0070bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0070bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0080bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0080bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0090bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0090bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0100bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0100bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0110bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0110bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0120bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0120bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0130bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0130bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0140bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0140bp_and_below'] = 0;
# data_values_indexed_scaled_logret['usdmxn_logret_-0150bp_and_above'] = 0; data_values_indexed_scaled_logret['usdmxn_logret_-0150bp_and_below'] = 0;

In [ ]:
# 02: Stratification: Initialize
for d_bp in xrange(-150,160,10): # -150:10:+150
  d = d_bp / 10000.00
  s_above = 'usdmxn_logret_' + '{0:+05d}'.format(d_bp) + 'bp_and_above'
  s_below = 'usdmxn_logret_' + '{0:+05d}'.format(d_bp) + 'bp_and_below'
  # print d
  # print s_above
  # print s_below
  data_values_indexed_scaled_logret[s_above] = 0;
  data_values_indexed_scaled_logret[s_below] = 0;

In [ ]:
# 02: Stratification: Populate results columns according to actual usdmxn returns (above or below each level)
for d_bp in xrange(-150,160,10): # -150:10:+150
  d = d_bp / 10000.00
  s_above = 'usdmxn_logret_' + '{0:+05d}'.format(d_bp) + 'bp_and_above'
  s_below = 'usdmxn_logret_' + '{0:+05d}'.format(d_bp) + 'bp_and_below'
  # print d
  # print s_above
  # print s_below
  data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] >= d, s_above] = 1
  data_values_indexed_scaled_logret.ix[data_values_indexed_scaled_logret[1] <  d, s_below] = 1


In [ ]:
data_values_indexed_scaled_logret.head()

In [ ]:
data_values_indexed_scaled_logret.describe()

## Model 1: Step 01.01: Remove data noise in a stratified way

In [ ]:
dvislr = data_values_indexed_scaled_logret
# dvislrnr = data_values_indexed_scaled_logret (noise removed)

# Example Filter: df[(df.A == 1) & (df.D == 6)]
dvislr_nr_m0010_z0000_p0010 = dvislr[ (dvislr['usdmxn_logret_-0010bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0010bp_and_above'] == 1) ] # remove data between [-0.10%, +0.10%], center at +0.00%
dvislr_nr_m0020_z0000_p0020 = dvislr[ (dvislr['usdmxn_logret_-0020bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0020bp_and_above'] == 1) ] # remove data between [-0.10%, +0.10%], center at +0.00%
dvislr_nr_m0030_z0000_p0030 = dvislr[ (dvislr['usdmxn_logret_-0030bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0030bp_and_above'] == 1) ] # remove data between [-0.10%, +0.10%], center at +0.00%
dvislr_nr_m0040_z0000_p0040 = dvislr[ (dvislr['usdmxn_logret_-0040bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0040bp_and_above'] == 1) ] # remove data between [-0.10%, +0.10%], center at +0.00%
dvislr_nr_m0050_z0000_p0050 = dvislr[ (dvislr['usdmxn_logret_-0050bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0050bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%
dvislr_nr_m0060_z0000_p0060 = dvislr[ (dvislr['usdmxn_logret_-0060bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0060bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%
dvislr_nr_m0070_z0000_p0070 = dvislr[ (dvislr['usdmxn_logret_-0070bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0070bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%
dvislr_nr_m0080_z0000_p0080 = dvislr[ (dvislr['usdmxn_logret_-0080bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0080bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%
dvislr_nr_m0090_z0000_p0090 = dvislr[ (dvislr['usdmxn_logret_-0090bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0090bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%
dvislr_nr_m0100_z0000_p0100 = dvislr[ (dvislr['usdmxn_logret_-0100bp_and_below'] == 1) | (dvislr['usdmxn_logret_+0100bp_and_above'] == 1) ] # remove data between [-0.50%, +0.50%], center at +0.00%

# check rows
print 'dvislr_nr_m0010_z0000_p0010 rows =' + str(len(dvislr_nr_m0010_z0000_p0010))
print 'dvislr_nr_m0020_z0000_p0020 rows =' + str(len(dvislr_nr_m0020_z0000_p0020))
print 'dvislr_nr_m0030_z0000_p0030 rows =' + str(len(dvislr_nr_m0030_z0000_p0030))
print 'dvislr_nr_m0040_z0000_p0040 rows =' + str(len(dvislr_nr_m0040_z0000_p0040))
print 'dvislr_nr_m0050_z0000_p0050 rows =' + str(len(dvislr_nr_m0050_z0000_p0050))
print 'dvislr_nr_m0060_z0000_p0060 rows =' + str(len(dvislr_nr_m0060_z0000_p0060))
print 'dvislr_nr_m0070_z0000_p0070 rows =' + str(len(dvislr_nr_m0070_z0000_p0070))
print 'dvislr_nr_m0080_z0000_p0080 rows =' + str(len(dvislr_nr_m0080_z0000_p0080))
print 'dvislr_nr_m0090_z0000_p0090 rows =' + str(len(dvislr_nr_m0090_z0000_p0090))
print 'dvislr_nr_m0100_z0000_p0100 rows =' + str(len(dvislr_nr_m0100_z0000_p0100))

# check rows results
# dvislr_nr_m0010_z0000_p0010 rows =3554
# dvislr_nr_m0020_z0000_p0020 rows =2904
# dvislr_nr_m0030_z0000_p0030 rows =2316
# dvislr_nr_m0040_z0000_p0040 rows =1837
# dvislr_nr_m0050_z0000_p0050 rows =1437
# dvislr_nr_m0060_z0000_p0060 rows =1135
# dvislr_nr_m0070_z0000_p0070 rows =893
# dvislr_nr_m0080_z0000_p0080 rows =697
# dvislr_nr_m0090_z0000_p0090 rows =559
# dvislr_nr_m0100_z0000_p0100 rows =439

## Model 1: Step 02: Split data into training, test

### training test data empty shell

In [ ]:
# training_test_data
# col 01-02 = 02 cols = indicators of results
# col 03-88 = 86 cols = inputs
training_test_data = pd.DataFrame(
  columns=[
    # 'usdmxn_logret_positive', 'usdmxn_logret_negative',
    # 'usdmxn_logret_positive_0050bp', 'usdmxn_logret_negative_0050bp',

    'bc_up', 'bc_dn',
    
    'usdmxn_logret_-0150bp_and_above', 'usdmxn_logret_-0150bp_and_below',
    'usdmxn_logret_-0140bp_and_above', 'usdmxn_logret_-0140bp_and_below',
    'usdmxn_logret_-0130bp_and_above', 'usdmxn_logret_-0130bp_and_below',
    'usdmxn_logret_-0120bp_and_above', 'usdmxn_logret_-0120bp_and_below',
    'usdmxn_logret_-0110bp_and_above', 'usdmxn_logret_-0110bp_and_below',
    'usdmxn_logret_-0100bp_and_above', 'usdmxn_logret_-0100bp_and_below',
    'usdmxn_logret_-0090bp_and_above', 'usdmxn_logret_-0090bp_and_below',
    'usdmxn_logret_-0080bp_and_above', 'usdmxn_logret_-0080bp_and_below',
    'usdmxn_logret_-0070bp_and_above', 'usdmxn_logret_-0070bp_and_below',
    'usdmxn_logret_-0060bp_and_above', 'usdmxn_logret_-0060bp_and_below',
    'usdmxn_logret_-0050bp_and_above', 'usdmxn_logret_-0050bp_and_below',
    'usdmxn_logret_-0040bp_and_above', 'usdmxn_logret_-0040bp_and_below',
    'usdmxn_logret_-0030bp_and_above', 'usdmxn_logret_-0030bp_and_below',
    'usdmxn_logret_-0020bp_and_above', 'usdmxn_logret_-0020bp_and_below',
    'usdmxn_logret_-0010bp_and_above', 'usdmxn_logret_-0010bp_and_below',
    'usdmxn_logret_+0000bp_and_above', 'usdmxn_logret_+0000bp_and_below',
    'usdmxn_logret_+0010bp_and_above', 'usdmxn_logret_+0010bp_and_below',
    'usdmxn_logret_+0020bp_and_above', 'usdmxn_logret_+0020bp_and_below',
    'usdmxn_logret_+0030bp_and_above', 'usdmxn_logret_+0030bp_and_below',
    'usdmxn_logret_+0040bp_and_above', 'usdmxn_logret_+0040bp_and_below',
    'usdmxn_logret_+0050bp_and_above', 'usdmxn_logret_+0050bp_and_below',
    'usdmxn_logret_+0060bp_and_above', 'usdmxn_logret_+0060bp_and_below',
    'usdmxn_logret_+0070bp_and_above', 'usdmxn_logret_+0070bp_and_below',
    'usdmxn_logret_+0080bp_and_above', 'usdmxn_logret_+0080bp_and_below',
    'usdmxn_logret_+0090bp_and_above', 'usdmxn_logret_+0090bp_and_below',
    'usdmxn_logret_+0100bp_and_above', 'usdmxn_logret_+0100bp_and_below',
    'usdmxn_logret_+0110bp_and_above', 'usdmxn_logret_+0110bp_and_below',
    'usdmxn_logret_+0120bp_and_above', 'usdmxn_logret_+0120bp_and_below',
    'usdmxn_logret_+0130bp_and_above', 'usdmxn_logret_+0130bp_and_below',
    'usdmxn_logret_+0140bp_and_above', 'usdmxn_logret_+0140bp_and_below',
    'usdmxn_logret_+0150bp_and_above', 'usdmxn_logret_+0150bp_and_below',

    '1','2','3','4','5','6','7','8','9','10',
    '11','12','13','14','15','16','17','18','19','20',
    '21','22','23','24','25','26','27','28','29','30',
    '31','32','33','34','35','36','37','38','39','40',
    '41','42','43','44','45','46','47','48','49','50',
    '51','52','53','54','55','56','57','58','59','60',
    '61','62','63','64','65','66','67','68','69','70',
    '71','72','73','74','75','76','77','78','79','80',
    '81','82','83','84','85','86',
  ])

training_test_data.head()

### Read data into variables

In [ ]:
# row 0      = header
# row 1-4279 = data
# NumRowsIncludeHeader = len(data_values_indexed_scaled_logret) = 4280 = 0-4279
# Start from row 7, so we can have up to (7-1) lookback days

# check rows results
# dvislr_nr_m0010_z0000_p0010 rows =3554
# dvislr_nr_m0020_z0000_p0020 rows =2904
# dvislr_nr_m0030_z0000_p0030 rows =2316
# dvislr_nr_m0040_z0000_p0040 rows =1837
# dvislr_nr_m0050_z0000_p0050 rows =1437
# dvislr_nr_m0060_z0000_p0060 rows =1135
# dvislr_nr_m0070_z0000_p0070 rows =893
# dvislr_nr_m0080_z0000_p0080 rows =697
# dvislr_nr_m0090_z0000_p0090 rows =559
# dvislr_nr_m0100_z0000_p0100 rows =439

#==========================================================================================
# Select the data
#==========================================================================================
# dvislr =  dvislr                          # default
# dvislr =  dvislr_nr_m0010_z0000_p0010
# dvislr =  dvislr_nr_m0020_z0000_p0020
# dvislr =  dvislr_nr_m0030_z0000_p0030
# dvislr =  dvislr_nr_m0040_z0000_p0040
# dvislr =  dvislr_nr_m0050_z0000_p0050
# dvislr =  dvislr_nr_m0060_z0000_p0060
# dvislr =  dvislr_nr_m0070_z0000_p0070
# dvislr =  dvislr_nr_m0080_z0000_p0080
# dvislr =  dvislr_nr_m0090_z0000_p0090
# dvislr =  dvislr_nr_m0100_z0000_p0100

for i in range(7, len(dvislr)): # [7, 4280) = [7,4279] 

  # 01: original
  usdmxn_logret_positive = data_values_indexed_scaled_logret['usdmxn_logret_positive'].ix[i]
  usdmxn_logret_negative = data_values_indexed_scaled_logret['usdmxn_logret_negative'].ix[i]

  # 02: stratification  
  usdmxn_logret_p0150bp_and_below = dvislr['usdmxn_logret_+0150bp_and_below'].ix[i]; usdmxn_logret_p0150bp_and_above = dvislr['usdmxn_logret_+0150bp_and_above'].ix[i];
  usdmxn_logret_p0140bp_and_below = dvislr['usdmxn_logret_+0140bp_and_below'].ix[i]; usdmxn_logret_p0140bp_and_above = dvislr['usdmxn_logret_+0140bp_and_above'].ix[i];
  usdmxn_logret_p0130bp_and_below = dvislr['usdmxn_logret_+0130bp_and_below'].ix[i]; usdmxn_logret_p0130bp_and_above = dvislr['usdmxn_logret_+0130bp_and_above'].ix[i];
  usdmxn_logret_p0120bp_and_below = dvislr['usdmxn_logret_+0120bp_and_below'].ix[i]; usdmxn_logret_p0120bp_and_above = dvislr['usdmxn_logret_+0120bp_and_above'].ix[i];
  usdmxn_logret_p0110bp_and_below = dvislr['usdmxn_logret_+0110bp_and_below'].ix[i]; usdmxn_logret_p0110bp_and_above = dvislr['usdmxn_logret_+0110bp_and_above'].ix[i];
  
  usdmxn_logret_p0100bp_and_below = dvislr['usdmxn_logret_+0100bp_and_below'].ix[i]; usdmxn_logret_p0100bp_and_above = dvislr['usdmxn_logret_+0100bp_and_above'].ix[i];
  usdmxn_logret_p0090bp_and_below = dvislr['usdmxn_logret_+0090bp_and_below'].ix[i]; usdmxn_logret_p0090bp_and_above = dvislr['usdmxn_logret_+0090bp_and_above'].ix[i];
  usdmxn_logret_p0080bp_and_below = dvislr['usdmxn_logret_+0080bp_and_below'].ix[i]; usdmxn_logret_p0080bp_and_above = dvislr['usdmxn_logret_+0080bp_and_above'].ix[i];
  usdmxn_logret_p0070bp_and_below = dvislr['usdmxn_logret_+0070bp_and_below'].ix[i]; usdmxn_logret_p0070bp_and_above = dvislr['usdmxn_logret_+0070bp_and_above'].ix[i];
  usdmxn_logret_p0060bp_and_below = dvislr['usdmxn_logret_+0060bp_and_below'].ix[i]; usdmxn_logret_p0060bp_and_above = dvislr['usdmxn_logret_+0060bp_and_above'].ix[i];

  usdmxn_logret_p0050bp_and_below = dvislr['usdmxn_logret_+0050bp_and_below'].ix[i]; usdmxn_logret_p0050bp_and_above = dvislr['usdmxn_logret_+0050bp_and_above'].ix[i];
  usdmxn_logret_p0040bp_and_below = dvislr['usdmxn_logret_+0040bp_and_below'].ix[i]; usdmxn_logret_p0040bp_and_above = dvislr['usdmxn_logret_+0040bp_and_above'].ix[i];
  usdmxn_logret_p0030bp_and_below = dvislr['usdmxn_logret_+0030bp_and_below'].ix[i]; usdmxn_logret_p0030bp_and_above = dvislr['usdmxn_logret_+0030bp_and_above'].ix[i];
  usdmxn_logret_p0020bp_and_below = dvislr['usdmxn_logret_+0020bp_and_below'].ix[i]; usdmxn_logret_p0020bp_and_above = dvislr['usdmxn_logret_+0020bp_and_above'].ix[i];
  usdmxn_logret_p0010bp_and_below = dvislr['usdmxn_logret_+0010bp_and_below'].ix[i]; usdmxn_logret_p0010bp_and_above = dvislr['usdmxn_logret_+0010bp_and_above'].ix[i];

  usdmxn_logret_p0000bp_and_below = dvislr['usdmxn_logret_+0000bp_and_below'].ix[i]; usdmxn_logret_p0000bp_and_above = dvislr['usdmxn_logret_+0000bp_and_above'].ix[i];

  usdmxn_logret_m0010bp_and_below = dvislr['usdmxn_logret_-0010bp_and_below'].ix[i]; usdmxn_logret_m0010bp_and_above = dvislr['usdmxn_logret_-0010bp_and_above'].ix[i];
  usdmxn_logret_m0020bp_and_below = dvislr['usdmxn_logret_-0020bp_and_below'].ix[i]; usdmxn_logret_m0020bp_and_above = dvislr['usdmxn_logret_-0020bp_and_above'].ix[i];
  usdmxn_logret_m0030bp_and_below = dvislr['usdmxn_logret_-0030bp_and_below'].ix[i]; usdmxn_logret_m0030bp_and_above = dvislr['usdmxn_logret_-0030bp_and_above'].ix[i];
  usdmxn_logret_m0040bp_and_below = dvislr['usdmxn_logret_-0040bp_and_below'].ix[i]; usdmxn_logret_m0040bp_and_above = dvislr['usdmxn_logret_-0040bp_and_above'].ix[i];
  usdmxn_logret_m0050bp_and_below = dvislr['usdmxn_logret_-0050bp_and_below'].ix[i]; usdmxn_logret_m0050bp_and_above = dvislr['usdmxn_logret_-0050bp_and_above'].ix[i];

  usdmxn_logret_m0060bp_and_below = dvislr['usdmxn_logret_-0060bp_and_below'].ix[i]; usdmxn_logret_m0060bp_and_above = dvislr['usdmxn_logret_-0060bp_and_above'].ix[i];
  usdmxn_logret_m0070bp_and_below = dvislr['usdmxn_logret_-0070bp_and_below'].ix[i]; usdmxn_logret_m0070bp_and_above = dvislr['usdmxn_logret_-0070bp_and_above'].ix[i];
  usdmxn_logret_m0080bp_and_below = dvislr['usdmxn_logret_-0080bp_and_below'].ix[i]; usdmxn_logret_m0080bp_and_above = dvislr['usdmxn_logret_-0080bp_and_above'].ix[i];
  usdmxn_logret_m0090bp_and_below = dvislr['usdmxn_logret_-0090bp_and_below'].ix[i]; usdmxn_logret_m0090bp_and_above = dvislr['usdmxn_logret_-0090bp_and_above'].ix[i];
  usdmxn_logret_m0100bp_and_below = dvislr['usdmxn_logret_-0100bp_and_below'].ix[i]; usdmxn_logret_m0100bp_and_above = dvislr['usdmxn_logret_-0100bp_and_above'].ix[i];

  usdmxn_logret_m0110bp_and_below = dvislr['usdmxn_logret_-0110bp_and_below'].ix[i]; usdmxn_logret_m0110bp_and_above = dvislr['usdmxn_logret_-0110bp_and_above'].ix[i];
  usdmxn_logret_m0120bp_and_below = dvislr['usdmxn_logret_-0120bp_and_below'].ix[i]; usdmxn_logret_m0120bp_and_above = dvislr['usdmxn_logret_-0120bp_and_above'].ix[i];
  usdmxn_logret_m0130bp_and_below = dvislr['usdmxn_logret_-0130bp_and_below'].ix[i]; usdmxn_logret_m0130bp_and_above = dvislr['usdmxn_logret_-0130bp_and_above'].ix[i];
  usdmxn_logret_m0140bp_and_below = dvislr['usdmxn_logret_-0140bp_and_below'].ix[i]; usdmxn_logret_m0140bp_and_above = dvislr['usdmxn_logret_-0140bp_and_above'].ix[i];
  usdmxn_logret_m0150bp_and_below = dvislr['usdmxn_logret_-0150bp_and_below'].ix[i]; usdmxn_logret_m0150bp_and_above = dvislr['usdmxn_logret_-0150bp_and_above'].ix[i];

  #==========================================================================================
  # Select 2 levels: up and dn
  #==========================================================================================
  bc_dn = usdmxn_logret_negative; bc_up = usdmxn_logret_positive; # 01: original
  #==========================================================================================  
#   bc_dn = usdmxn_logret_m0010bp_and_below; bc_up = usdmxn_logret_p0010bp_and_above;
#   bc_dn = usdmxn_logret_m0020bp_and_below; bc_up = usdmxn_logret_p0020bp_and_above;
#   bc_dn = usdmxn_logret_m0030bp_and_below; bc_up = usdmxn_logret_p0030bp_and_above;
#   bc_dn = usdmxn_logret_m0040bp_and_below; bc_up = usdmxn_logret_p0040bp_and_above;
#   bc_dn = usdmxn_logret_m0050bp_and_below; bc_up = usdmxn_logret_p0050bp_and_above;
#   bc_dn = usdmxn_logret_m0060bp_and_below; bc_up = usdmxn_logret_p0060bp_and_above;
#   bc_dn = usdmxn_logret_m0070bp_and_below; bc_up = usdmxn_logret_p0070bp_and_above;
#   bc_dn = usdmxn_logret_m0080bp_and_below; bc_up = usdmxn_logret_p0080bp_and_above;
#   bc_dn = usdmxn_logret_m0090bp_and_below; bc_up = usdmxn_logret_p0090bp_and_above;
#   bc_dn = usdmxn_logret_m0100bp_and_below; bc_up = usdmxn_logret_p0100bp_and_above;
  
  #   v_001_Tm005 = data_values_indexed_scaled_logret['1'].ix[i-5]  # lookback 1d

  v_001_Tm000 = dvislr[1].ix[i];  v_002_Tm000 = dvislr[2].ix[i];  v_003_Tm000 = dvislr[3].ix[i];  v_004_Tm000 = dvislr[4].ix[i];  v_005_Tm000 = dvislr[5].ix[i];
  v_006_Tm000 = dvislr[6].ix[i];  v_007_Tm000 = dvislr[7].ix[i];  v_008_Tm000 = dvislr[8].ix[i];  v_009_Tm000 = dvislr[9].ix[i];  v_010_Tm000 = dvislr[10].ix[i]

  v_011_Tm000 = dvislr[11].ix[i]; v_012_Tm000 = dvislr[12].ix[i]; v_013_Tm000 = dvislr[13].ix[i]; v_014_Tm000 = dvislr[14].ix[i]; v_015_Tm000 = dvislr[15].ix[i];
  v_016_Tm000 = dvislr[16].ix[i]; v_017_Tm000 = dvislr[17].ix[i]; v_018_Tm000 = dvislr[18].ix[i]; v_019_Tm000 = dvislr[19].ix[i]; v_020_Tm000 = dvislr[20].ix[i];
  
  v_021_Tm000 = dvislr[21].ix[i]; v_022_Tm000 = dvislr[22].ix[i]; v_023_Tm000 = dvislr[23].ix[i]; v_024_Tm000 = dvislr[24].ix[i]; v_025_Tm000 = dvislr[25].ix[i];
  v_026_Tm000 = dvislr[26].ix[i]; v_027_Tm000 = dvislr[27].ix[i]; v_028_Tm000 = dvislr[28].ix[i]; v_029_Tm000 = dvislr[29].ix[i]; v_030_Tm000 = dvislr[30].ix[i];

  v_031_Tm000 = dvislr[31].ix[i]; v_032_Tm000 = dvislr[32].ix[i]; v_033_Tm000 = dvislr[33].ix[i]; v_034_Tm000 = dvislr[34].ix[i]; v_035_Tm000 = dvislr[35].ix[i];
  v_036_Tm000 = dvislr[36].ix[i]; v_037_Tm000 = dvislr[37].ix[i]; v_038_Tm000 = dvislr[38].ix[i]; v_039_Tm000 = dvislr[39].ix[i]; v_040_Tm000 = dvislr[40].ix[i];

  v_041_Tm000 = dvislr[41].ix[i]; v_042_Tm000 = dvislr[42].ix[i]; v_043_Tm000 = dvislr[43].ix[i]; v_044_Tm000 = dvislr[44].ix[i]; v_045_Tm000 = dvislr[45].ix[i];
  v_046_Tm000 = dvislr[46].ix[i]; v_047_Tm000 = dvislr[47].ix[i]; v_048_Tm000 = dvislr[48].ix[i]; v_049_Tm000 = dvislr[49].ix[i]; v_050_Tm000 = dvislr[50].ix[i];

  v_051_Tm000 = dvislr[51].ix[i]; v_052_Tm000 = dvislr[52].ix[i]; v_053_Tm000 = dvislr[53].ix[i]; v_054_Tm000 = dvislr[54].ix[i]; v_055_Tm000 = dvislr[55].ix[i];
  v_056_Tm000 = dvislr[56].ix[i]; v_057_Tm000 = dvislr[57].ix[i]; v_058_Tm000 = dvislr[58].ix[i]; v_059_Tm000 = dvislr[59].ix[i]; v_060_Tm000 = dvislr[60].ix[i];

  v_061_Tm000 = dvislr[61].ix[i]; v_062_Tm000 = dvislr[62].ix[i]; v_063_Tm000 = dvislr[63].ix[i]; v_064_Tm000 = dvislr[64].ix[i]; v_065_Tm000 = dvislr[65].ix[i];
  v_066_Tm000 = dvislr[66].ix[i]; v_067_Tm000 = dvislr[67].ix[i]; v_068_Tm000 = dvislr[68].ix[i]; v_069_Tm000 = dvislr[69].ix[i]; v_070_Tm000 = dvislr[70].ix[i];

  v_071_Tm000 = dvislr[71].ix[i]; v_072_Tm000 = dvislr[72].ix[i]; v_073_Tm000 = dvislr[73].ix[i]; v_074_Tm000 = dvislr[74].ix[i]; v_075_Tm000 = dvislr[75].ix[i];
  v_076_Tm000 = dvislr[76].ix[i]; v_077_Tm000 = dvislr[77].ix[i]; v_078_Tm000 = dvislr[78].ix[i]; v_079_Tm000 = dvislr[79].ix[i]; v_080_Tm000 = dvislr[80].ix[i];

  v_081_Tm000 = dvislr[81].ix[i]; v_082_Tm000 = dvislr[82].ix[i]; v_083_Tm000 = dvislr[83].ix[i]; v_084_Tm000 = dvislr[84].ix[i]; v_085_Tm000 = dvislr[85].ix[i];
  v_086_Tm000 = dvislr[86].ix[i];

  # training_test_data: append data
  training_test_data = training_test_data.append(
    {
#       'usdmxn_logret_positive':usdmxn_logret_positive,
#       'usdmxn_logret_negative':usdmxn_logret_negative,
#       'usdmxn_logret_positive_0050bp':usdmxn_logret_positive_0050bp,
#       'usdmxn_logret_negative_0050bp':usdmxn_logret_negative_0050bp,
      
      'bc_up':bc_up,
      'bc_dn':bc_dn,
      
      '1':v_001_Tm000,  '2':v_002_Tm000, '3':v_003_Tm000, '4':v_004_Tm000, '5':v_005_Tm000,
      '6':v_006_Tm000,  '7':v_007_Tm000, '8':v_008_Tm000, '9':v_009_Tm000, '10':v_010_Tm000,

      '11':v_011_Tm000, '12':v_012_Tm000, '13':v_013_Tm000, '14':v_014_Tm000, '15':v_015_Tm000,
      '16':v_016_Tm000, '17':v_017_Tm000, '18':v_018_Tm000, '19':v_019_Tm000, '20':v_020_Tm000,

      '21':v_021_Tm000, '22':v_022_Tm000, '23':v_023_Tm000, '24':v_024_Tm000, '25':v_025_Tm000,
      '26':v_026_Tm000, '27':v_027_Tm000, '28':v_028_Tm000, '29':v_029_Tm000, '30':v_030_Tm000,

      '31':v_031_Tm000, '32':v_032_Tm000, '33':v_033_Tm000, '34':v_034_Tm000, '35':v_035_Tm000,
      '36':v_036_Tm000, '37':v_037_Tm000, '38':v_038_Tm000, '39':v_039_Tm000, '40':v_040_Tm000,

      '41':v_041_Tm000, '42':v_042_Tm000, '43':v_043_Tm000, '44':v_044_Tm000, '45':v_045_Tm000,
      '46':v_046_Tm000, '47':v_047_Tm000, '48':v_048_Tm000, '49':v_049_Tm000, '50':v_050_Tm000,

      '51':v_051_Tm000, '52':v_052_Tm000, '53':v_053_Tm000, '54':v_054_Tm000, '55':v_055_Tm000,
      '56':v_056_Tm000, '57':v_057_Tm000, '58':v_058_Tm000, '59':v_059_Tm000, '60':v_060_Tm000,

      '61':v_061_Tm000, '62':v_062_Tm000, '63':v_063_Tm000, '64':v_064_Tm000, '65':v_065_Tm000,
      '66':v_066_Tm000, '67':v_067_Tm000, '68':v_068_Tm000, '69':v_069_Tm000, '70':v_070_Tm000,

      '71':v_071_Tm000, '72':v_072_Tm000, '73':v_073_Tm000, '74':v_074_Tm000, '75':v_075_Tm000,
      '76':v_076_Tm000, '77':v_077_Tm000, '78':v_078_Tm000, '79':v_079_Tm000, '80':v_080_Tm000,
      
      '81':v_081_Tm000, '82':v_082_Tm000, '83':v_083_Tm000, '84':v_084_Tm000, '85':v_085_Tm000,
      '86':v_086_Tm000,
      
    },
    ignore_index=True)
  
# data_values_indexed_scaled_logret: row [7,4279] 
# training_test_data               : row [0,4272] = 4273 rows
# training_test_data: col 01-02 = 02 cols = binary outputs
# training_test_data: col 03-88 = 86 cols = inputs  


In [ ]:
# training_test_data.describe()
training_test_data[['bc_up','bc_dn']].describe()

# check rows results
# dvislr_nr_m0010_z0000_p0010 rows =3554 - 7 = 3547
# dvislr_nr_m0020_z0000_p0020 rows =2904 - 7 = 2897
# dvislr_nr_m0030_z0000_p0030 rows =2316 - 7 = 2309
# dvislr_nr_m0040_z0000_p0040 rows =1837 - 7 = 1830
# dvislr_nr_m0050_z0000_p0050 rows =1437 - 7 = 1430
# dvislr_nr_m0060_z0000_p0060 rows =1135 - 7 = 1128
# dvislr_nr_m0070_z0000_p0070 rows = 893 - 7 =  886
# dvislr_nr_m0080_z0000_p0080 rows = 697 - 7 =  690
# dvislr_nr_m0090_z0000_p0090 rows = 559 - 7 =  552
# dvislr_nr_m0100_z0000_p0100 rows = 439 - 7 =  432

### create the training and test data.

In [ ]:
# 86 input columns
predictors_tf = training_test_data[training_test_data.columns[0+2+(31*2):0+2+(31*2)+86]]
predictors_tf.describe()

In [ ]:
# 2 output columns
classes_tf = training_test_data[training_test_data.columns[0:2]] # col 0, 1
# classes_tf = training_test_data[training_test_data.columns[2:4]] # col 2, 3
classes_tf.describe()

In [ ]:
# Split: train data = 80%
training_set_size = int(len(training_test_data) * 0.8)
print 'training_set_size=' + str(training_set_size )
# Split: test data = 20%
test_set_size = len(training_test_data) - training_set_size
print 'test_set_size=' + str(test_set_size )

# noise remove stratification: [-0010,+0010]: training_set_size=2837, test_set_size=710
# noise remove stratification: [-0020,+0020]: training_set_size=2317, test_set_size=580
# noise remove stratification: [-0030,+0030]: training_set_size=1847, test_set_size=462
# noise remove stratification: [-0040,+0040]: training_set_size=1464, test_set_size=366
# noise remove stratification: [-0050,+0050]: training_set_size=1144, test_set_size=286
# noise remove stratification: [-0060,+0060]: training_set_size=902,  test_set_size=226
# noise remove stratification: [-0070,+0070]: training_set_size=708,  test_set_size=178
# noise remove stratification: [-0080,+0080]: training_set_size=552,  test_set_size=138
# noise remove stratification: [-0090,+0090]: training_set_size=441,  test_set_size=111
# noise remove stratification: [-0100,+0100]: training_set_size=345,  test_set_size=87


## Split into training and test data

In [ ]:
training_predictors_tf = predictors_tf[:training_set_size]
training_classes_tf = classes_tf[:training_set_size]
test_predictors_tf = predictors_tf[training_set_size:]
test_classes_tf = classes_tf[training_set_size:]


Define some metrics here to evaluate the models.

* [Precision](https://en.wikipedia.org/wiki/Precision_and_recall#Precision) -  The ability of the classifier not to label as positive a sample that is negative.
* [Recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) - The ability of the classifier to find all the positive samples.
* [F1 Score](https://en.wikipedia.org/wiki/F1_score) - A weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.
* Accuracy - The percentage correctly predicted in the test data.

In [ ]:
def tf_confusion_metrics(model, actual_classes, session, feed_dict):
  predictions = tf.argmax(model, 1)
  actuals = tf.argmax(actual_classes, 1)

  ones_like_actuals = tf.ones_like(actuals)
  zeros_like_actuals = tf.zeros_like(actuals)
  ones_like_predictions = tf.ones_like(predictions)
  zeros_like_predictions = tf.zeros_like(predictions)

  tp_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, ones_like_actuals), 
        tf.equal(predictions, ones_like_predictions)
      ), 
      "float"
    )
  )

  tn_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, zeros_like_actuals), 
        tf.equal(predictions, zeros_like_predictions)
      ), 
      "float"
    )
  )

  fp_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, zeros_like_actuals), 
        tf.equal(predictions, ones_like_predictions)
      ), 
      "float"
    )
  )

  fn_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, ones_like_actuals), 
        tf.equal(predictions, zeros_like_predictions)
      ), 
      "float"
    )
  )

  tp, tn, fp, fn = \
    session.run(
      [tp_op, tn_op, fp_op, fn_op], 
      feed_dict
    )

  tpr = float(tp)/(float(tp) + float(fn))
  fpr = float(fp)/(float(tp) + float(fn))

  accuracy = (float(tp) + float(tn))/(float(tp) + float(fp) + float(fn) + float(tn))

  recall = tpr
  precision = float(tp)/(float(tp) + float(fp))
  
  f1_score = (2 * (precision * recall)) / (precision + recall)
  
  print 'Precision = ', precision
  print 'Recall = ', recall
  print 'F1 Score = ', f1_score
  print 'Accuracy = ', accuracy

### Binary classification with TensorFlow

Now, get some tensors flowing. The model is binary classification expressed in TensorFlow.

In [ ]:
# tensorflow session
sess = tf.Session()
# Define variables for the number of predictors and number of classes to remove magic numbers from our code.
num_predictors = len(training_predictors_tf.columns) # 24-6=18 in the default case
num_classes = len(training_classes_tf.columns) # 2 in the default case
print 'num_predictors=' + str(num_predictors)
print 'num_classes=' + str(num_classes)
# Define placeholders for the data we feed into the process - feature data and actual classes.
feature_data = tf.placeholder("float", [None, num_predictors])
actual_classes = tf.placeholder("float", [None, num_classes])
# Define a matrix of weights and initialize it with some small random values.
weights = tf.Variable(tf.truncated_normal([num_predictors, num_classes], stddev=0.0001))
biases = tf.Variable(tf.ones([num_classes]))
# Define our model...
# Here we take a softmax regression of the product of our feature data and weights.
model = tf.nn.softmax(tf.matmul(feature_data, weights) + biases)

# Define a cost function (we're using the cross entropy).
cost = -tf.reduce_sum(actual_classes*tf.log(model))

# Define a training step...
# Here we use gradient descent with a learning rate of 0.01 using the cost function we just defined.
training_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

init = tf.initialize_all_variables()
sess.run(init)

In [ ]:
# BEFORE the model has been run, ie. not yet trained
# display weights
w_86_2 = sess.run(weights)    
# print w_86_2 

# Expect something like this (small numbers e-05): 
# [[ -6.66152628e-05  -6.49469657e-05]
#  [  2.86067752e-05   3.27789494e-05]
#  [  4.73170294e-05   1.85125769e-04]
#  [ -6.42955492e-05   4.16024632e-05]
#  [ -1.54488771e-05  -2.10903818e-05]
#  [  4.62506796e-05  -2.98624745e-05]
#  [ -5.35508079e-05  -1.30856875e-04]
#  [ -1.70812025e-04   1.33106529e-04]
#  [ -1.45097118e-04  -1.80459567e-04]
#  [  1.02448161e-04   8.27739277e-05]
#  [ -1.33277281e-04  -4.04360726e-05]
#  [ -1.36186543e-04   8.62382149e-05]
#  [  2.80324894e-05  -2.20580205e-05]
#  [ -4.52000713e-05   2.54859442e-05]
#  [ -6.20259525e-05  -6.95227573e-05]
#  [  1.68935469e-04  -4.03221093e-05]
#  [ -9.51008842e-05   4.10227585e-05]
#  [ -5.90909331e-05  -9.20566745e-05]]

We'll train our model in the following snippet. The approach of TensorFlow to executing graph operations allows fine-grained control over the process. Any operation you provide to the session as part of the run operation will be executed and the results returned. You can provide a list of multiple operations.

You'll train the model over 30,000 iterations using the full dataset each time. Every thousandth iteration we'll assess the accuracy of the model on the training data to assess progress.

In [ ]:
correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(actual_classes, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

for i in range(1, 30001):
  sess.run(
    training_step, 
    feed_dict={
      feature_data: training_predictors_tf.values, 
      actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
    }
  )
  if i%5000 == 0:
    print i, sess.run(
      accuracy,
      feed_dict={
        feature_data: training_predictors_tf.values, 
        actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
      }
    )
    
# Expect:
# 5000 0.603862
# 10000 0.630778
# 15000 0.639555
# 20000 0.654184
# 25000 0.660035
# 30000 0.666764

# After using < -0.5% and > +0.5%
# 5000 0.928906
# 10000 0.928906
# 15000 0.928906
# 20000 0.928906
# 25000 0.928906
# 30000 0.928906

# Noise Removal: [-0010, +0010]
# 5000 0.617201
# 10000 0.64399
# 15000 0.665139
# 20000 0.683468
# 25000 0.688403
# 30000 0.691928

# Noise Removal: [-0020, +0020]
# 5000 0.625809
# 10000 0.6612
# 15000 0.680622
# 20000 0.695727
# 25000 0.708675
# 30000 0.712128

# Noise Removal: [-0030, +0030]
# 5000 0.635084
# 10000 0.677315
# 15000 0.701678
# 20000 0.714672
# 25000 0.729291
# 30000 0.738495

# Noise Removal: [-0040, +0040]
# 5000 0.645492
# 10000 0.689891
# 15000 0.717213
# 20000 0.734973
# 25000 0.748634
# 30000 0.758197

# Noise Removal: [-0050, +0050]
# 5000 0.653846
# 10000 0.702797
# 15000 0.730769
# 20000 0.740385
# 25000 0.75
# 30000 0.768357

# Noise Removal: [-0060, +0060]
# 5000 0.672949
# 10000 0.72173
# 15000 0.750554
# 20000 0.761641
# 25000 0.783814
# 30000 0.792683

# Noise Removal: [-0070, +0070]
# 5000 0.673729
# 10000 0.735876
# 15000 0.771186
# 20000 0.789548
# 25000 0.809322
# 30000 0.817797

# Noise Removal: [-0080, +0080]
# 5000 0.692029
# 10000 0.748188
# 15000 0.780797
# 20000 0.815217
# 25000 0.826087
# 30000 0.833333

# Noise Removal: [-0090, +0090]
# 5000 0.696145
# 10000 0.750567
# 15000 0.773243
# 20000 0.795918
# 25000 0.811791
# 30000 0.816327

# Noise Removal: [-0100, +0100]
# 5000 0.718841
# 10000 0.75942
# 15000 0.788406
# 20000 0.808696
# 25000 0.823188
# 30000 0.84058


Using all data:
Accuracy  66.6% on training data
That is OK, better than random.

After noise removed, We get 77 % accuracy on training data.

## Accuracy on test data

In [ ]:
feed_dict= {
  feature_data: test_predictors_tf.values,
  actual_classes: test_classes_tf.values.reshape(len(test_classes_tf.values), 2)
}

tf_confusion_metrics(model, actual_classes, sess, feed_dict)

# Precision =  0.541176470588
# Recall =  0.555555555556
# F1 Score =  0.548271752086
# Accuracy =  0.556725146199

# After removing noise
# Precision =  0.684210526316
# Recall =  0.534246575342
# F1 Score =  0.6
# Accuracy =  0.636363636364

# Noise Removal: [-0010, +0010]
# Precision =  0.577235772358
# Recall =  0.605113636364
# F1 Score =  0.590846047157
# Accuracy =  0.584507042254

# Noise Removal: [-0020, +0020]
# Precision =  0.588850174216
# Recall =  0.603571428571
# F1 Score =  0.596119929453
# Accuracy =  0.605172413793

# Noise Removal: [-0030, +0030]
# Precision =  0.616740088106
# Recall =  0.619469026549
# F1 Score =  0.618101545254
# Accuracy =  0.625541125541

# Noise Removal: [-0040, +0040]
# Precision =  0.622093023256
# Recall =  0.58152173913
# F1 Score =  0.601123595506
# Accuracy =  0.612021857923

# Noise Removal: [-0050, +0050]
# Precision =  0.684210526316
# Recall =  0.534246575342
# F1 Score =  0.6
# Accuracy =  0.636363636364

# Noise Removal: [-0060, +0060]
# Precision =  0.644736842105
# Recall =  0.453703703704
# F1 Score =  0.532608695652
# Accuracy =  0.619469026549

# Noise Removal: [-0070, +0070]
# Precision =  0.666666666667
# Recall =  0.47619047619
# F1 Score =  0.555555555556
# Accuracy =  0.640449438202

# Noise Removal: [-0080, +0080]
# Precision =  0.6
# Recall =  0.406779661017
# F1 Score =  0.484848484848
# Accuracy =  0.630434782609

# Noise Removal: [-0090, +0090]
# Precision =  0.478260869565
# Recall =  0.261904761905
# F1 Score =  0.338461538462
# Accuracy =  0.612612612613

# Noise Removal: [-0100, +0100]
# Precision =  0.588235294118
# Recall =  0.37037037037
# F1 Score =  0.454545454545
# Accuracy =  0.724137931034



# check rows results                         numrows   AccuOnTrainData    AccuOnTestData
# dvislr_nr_m0010_z0000_p0010 rows =3554 - 7 = 3547    # 30000 0.691928   # Accuracy =  0.584507042254
# dvislr_nr_m0020_z0000_p0020 rows =2904 - 7 = 2897    # 30000 0.712128   # Accuracy =  0.605172413793
# dvislr_nr_m0030_z0000_p0030 rows =2316 - 7 = 2309    # 30000 0.738495   # Accuracy =  0.625541125541
# dvislr_nr_m0040_z0000_p0040 rows =1837 - 7 = 1830    # 30000 0.758197   # Accuracy =  0.612021857923
# dvislr_nr_m0050_z0000_p0050 rows =1437 - 7 = 1430    # 30000 0.768357   # Accuracy =  0.636363636364
# dvislr_nr_m0060_z0000_p0060 rows =1135 - 7 = 1128    # 30000 0.792683   # Accuracy =  0.619469026549
# dvislr_nr_m0070_z0000_p0070 rows = 893 - 7 =  886    # 30000 0.817797   # Accuracy =  0.640449438202
# dvislr_nr_m0080_z0000_p0080 rows = 697 - 7 =  690    # 30000 0.833333   # Accuracy =  0.630434782609
# dvislr_nr_m0090_z0000_p0090 rows = 559 - 7 =  552    # 30000 0.816327   # Accuracy =  0.612612612613
# dvislr_nr_m0100_z0000_p0100 rows = 439 - 7 =  432    # 30000 0.84058    # Accuracy =  0.724137931034




Using all data, Accuracy 55.6% on test data.
After removing noise, Accuracy 64% on test data.

In [ ]:
# AFTER the model has been run, ie. trained
# display weights
w_86_2 = sess.run(weights)    
print w_86_2 

In [ ]:
# response = client.put_file('/output.csv', str(pd.DataFrame(npa).to_csv()), overwrite=True)
# response = client.put_file('/output.csv', str(               df.to_csv()), overwrite=True)
response = client.put_file('/m01_BC_w_86_2.csv', str(pd.DataFrame(w_86_2).to_csv()), overwrite=True)
print "uploaded:", response

In [ ]:
# grab a sample input row from the test set
sample_input_1_86 = test_predictors_tf[:10]
# HTML(pd.DataFrame(sample_input_1_86).to_html())
sample_input_1_86

In [ ]:
sample_input_1_86.describe()

In [ ]:
# use the weights from our model to make a prediction
output_1_2 = np.dot(sample_input_1_86, w_86_2) > 0
print output_1_2
usdmxn_up = output_1_2[0,0]
usdmxn_dn = output_1_2[0,1]
if usdmxn_up >= usdmxn_dn:
  usdmxn_pred = 1 # up
else:
  usdmxn_pred = -1 # down
print 'usdmxn_pred = '+ str(usdmxn_pred)

# Model 2: Neural Network (NN): Feed-Forward with 2 Hidden Layers

You'll now build a proper feed-forward neural net with two hidden layers.

In [ ]:
# tensorflow session new
sess1 = tf.Session()

In [ ]:
# inputs
num_predictors = len(training_predictors_tf.columns)
print 'num_predictors=' + str(num_predictors)

# outputs
num_classes = len(training_classes_tf.columns)
print 'num_classes=' + str(num_classes)

In [ ]:
feature_data = tf.placeholder("float", [None, num_predictors])
actual_classes = tf.placeholder("float", [None, 2])

In [ ]:
# layer 1
i_01_in = 86
i_01_out = 100
weights1 = tf.Variable(tf.truncated_normal([i_01_in, i_01_out], stddev=0.0001))
biases1 = tf.Variable(tf.ones([i_01_out]))

In [ ]:
# layer 2
i_02_in = i_01_out
i_02_out = 25
weights2 = tf.Variable(tf.truncated_normal([i_02_in, i_02_out], stddev=0.0001))
biases2 = tf.Variable(tf.ones([i_02_out]))

In [ ]:
# layer 3
i_03_in = i_02_out
i_03_out = 2
weights3 = tf.Variable(tf.truncated_normal([i_03_in, i_03_out], stddev=0.0001))
biases3 = tf.Variable(tf.ones([i_03_out]))

In [ ]:
# neural net
hidden_layer_0 = feature_data
hidden_layer_1 = tf.nn.relu(tf.matmul(hidden_layer_0, weights1) + biases1)
hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1, weights2) + biases2)
hidden_layer_3 = tf.matmul(hidden_layer_2, weights3) + biases3

model = tf.nn.softmax(hidden_layer_3)

cost = -tf.reduce_sum(actual_classes*tf.log(model))

train_op1 = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

init = tf.initialize_all_variables()
sess1.run(init)

In [ ]:
# BEFORE the model has been run, ie. trained
nn_e1 = sess1.run(biases1)    
# HTML(pd.DataFrame(nn_e1).transpose().to_html())
pd.DataFrame(nn_e1).transpose()


In [ ]:
# BEFORE the model has been run, ie. trained
nn_w1 = sess1.run(weights1)    
# HTML(pd.DataFrame(nn_w1).to_html())
pd.DataFrame(nn_w1)


In [ ]:
# BEFORE the model has been run, ie. trained
nn_e2 = sess1.run(biases2)    
# HTML(pd.DataFrame(nn_e2).transpose().to_html())
pd.DataFrame(nn_e2).transpose()


In [ ]:
# BEFORE the model has been run, ie. trained
# display weights
nn_w2 = sess1.run(weights2)    
# HTML(pd.DataFrame(nn_w2).to_html())
pd.DataFrame(nn_w2)


In [ ]:
# BEFORE the model has been run, ie. trained
nn_e3 = sess1.run(biases3)    
# HTML(pd.DataFrame(nn_e3).transpose().to_html())
pd.DataFrame(nn_e3).transpose()


In [ ]:
# BEFORE the model has been run, ie. trained
# display weights
nn_w3 = sess1.run(weights3)    
# HTML(pd.DataFrame(nn_w3).to_html())
pd.DataFrame(nn_w3)


Again, you'll train the model over 10,000 iterations using the full dataset each time. Every thousandth iteration, you'll assess the accuracy of the model on the training data to assess progress.

In [ ]:
correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(actual_classes, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
num_iterations = 5001
num_iterations_until_display = 1000

for i in range(1, num_iterations):
  sess1.run(
    train_op1, 
    feed_dict={
      feature_data: training_predictors_tf.values, 
      actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
    }
  )
  if i%num_iterations_until_display == 0:
    print i, sess1.run(
      accuracy,
      feed_dict={
        feature_data: training_predictors_tf.values, 
        actual_classes: training_classes_tf.values.reshape(len(training_classes_tf.values), 2)
      }
    )
  
# Original
# 1000 0.676126
# 2000 0.768578
# 3000 0.83031
# 4000 0.891164
# 5000 0.93739
# 6000 0.956115
# 7000 0.972206
# 8000 0.978935
# 9000 0.985079
# 10000 0.987127

# Original
# 1000 0.685489
# 2000 0.777063
# 3000 0.84055
# 4000 0.902867
# 5000 0.941486
# 6000 0.957578
# 7000 0.974254
# 8000 0.980105
# 9000 0.985079
# 10000 0.987712


A significant improvement in accuracy with the training data shows that the hidden layers are adding additional capacity for learning to the model.

Looking at precision, recall, and accuracy, you can see a measurable improvement in performance, but certainly not a [step function](https://wikipedia.org/wiki/Step_function). This indicates that we're likely reaching the limits of this relatively simple feature set.

In [ ]:
feed_dict= {
  feature_data: test_predictors_tf.values,
  actual_classes: test_classes_tf.values.reshape(len(test_classes_tf.values), 2)
}

tf_confusion_metrics(model, actual_classes, sess1, feed_dict)

# Original
# Precision =  0.934731934732
# Recall =  0.968599033816
# F1 Score =  0.951364175563
# Accuracy =  0.952046783626


In [ ]:
# AFTER the model has been run, ie. trained
nn_e1 = sess1.run(biases1)    
HTML(pd.DataFrame(nn_e1).transpose().to_html())
# pd.DataFrame(nn_e1).transpose()


In [ ]:
# response = client.put_file('/output.csv', str(pd.DataFrame(npa).to_csv()), overwrite=True)
# response = client.put_file('/output.csv', str(               df.to_csv()), overwrite=True)
response = client.put_file('/m02_NN_nn_e1.csv', str(pd.DataFrame(nn_e1).to_csv()), overwrite=True)
print "uploaded:", response

In [ ]:
# AFTER the model has been run, ie. trained
# display weights
nn_w1 = sess1.run(weights1)    
HTML(pd.DataFrame(nn_w1).to_html())
# pd.DataFrame(nn_w1)


In [ ]:
# AFTER the model has been run, ie. trained
nn_e2 = sess1.run(biases2)    
HTML(pd.DataFrame(nn_e2).transpose().to_html())


In [ ]:
# AFTER the model has been run, ie. trained
# display weights
nn_w2 = sess1.run(weights2)    
HTML(pd.DataFrame(nn_w2).to_html())


In [ ]:
# AFTER the model has been run, ie. trained
nn_e3 = sess1.run(biases3)    
HTML(pd.DataFrame(nn_e3).transpose().to_html())


In [ ]:
# AFTER the model has been run, ie. trained
# display weights
nn_w3 = sess1.run(weights3)    
HTML(pd.DataFrame(nn_w3).to_html())


In [ ]:
# response = client.put_file('/output.csv', str(pd.DataFrame(npa).to_csv()), overwrite=True)
# response = client.put_file('/output.csv', str(               df.to_csv()), overwrite=True)
response = client.put_file('/m02_NN_nn_e1.csv', str(pd.DataFrame(nn_e1).to_csv()), overwrite=True); print "uploaded:", response
response = client.put_file('/m02_NN_nn_w1.csv', str(pd.DataFrame(nn_w1).to_csv()), overwrite=True); print "uploaded:", response
response = client.put_file('/m02_NN_nn_e2.csv', str(pd.DataFrame(nn_e2).to_csv()), overwrite=True); print "uploaded:", response
response = client.put_file('/m02_NN_nn_w2.csv', str(pd.DataFrame(nn_w2).to_csv()), overwrite=True); print "uploaded:", response
response = client.put_file('/m02_NN_nn_e3.csv', str(pd.DataFrame(nn_e3).to_csv()), overwrite=True); print "uploaded:", response
response = client.put_file('/m02_NN_nn_w3.csv', str(pd.DataFrame(nn_w3).to_csv()), overwrite=True); print "uploaded:", response

In [ ]:
# # AFTER the model has been run, ie. trained
# nn_hl3 = sess1.run(hidden_layer_3)    
# HTML(pd.DataFrame(nn_hl3).to_html())


## Conclusion

You've covered a lot of ground. You moved from sourcing five years of financial time-series data, to munging that data into a more suitable form. You explored and visualized that data with exploratory data analysis and then decided on a machine learning model and the features for that model. You engineered those features, built a binary classifier in TensorFlow, and analyzed its performance. You built a feed forward neural net with two hidden layers in TensorFlow and analyzed its performance.

How did the technology fare? It should take most people 1.5 to 3 hours to extract the juice from this solution, and none of that time is spent waiting for infrastructure or software; it's spent reading and thinking. In many organizations, it can take anywhere from days to months to do this sort of data analysis, depending on whether you need to procure any hardware. And you didn't need to do anything with infrastructure or additional software. Rather, you used a web-based console to direct GCP to set up systems on your behalf, which it did—fully managed, maintained, and supported—freeing you up to spend your time analyzing. 

It was also cost effective. If you took your time with this solution and spent three hours to go through it, the cost would be a few pennies. 

Cloud Datalab worked admirably, too. iPython/Jupyter has always been a great platform for interactive, iterative work and a fully-managed version of that platform on GCP, with connectors to other GCP technologies such as BigQuery and Google Cloud Storage, is a force multiplier for your analysis needs.  If you haven't used iPython before, this solution might have been eye opening, for you. If you're already familiar with iPython, then you'll love the connectors to other GCP technologies.

Of course, R and Matlab are popular tools in machine learning, and we've made no mention either in this solution. Neither R nor Matlab are available as managed services on GCP. Both can be hosted in GCP and accessed through a cloud-friendly, web frontend.

TensorFlow is a special piece of technology. It is expressive, performs well, and comes with the weight of Google's machine learning history and expertise to back it up and support it. We've only scratched the surface, but you can already see that within a handful of lines of code we've been able to write two models. Neither of them is cutting edge, by design, but neither of them is trivial either. With some additional tuning they would suit a whole spectrum of machine learning tasks. 

Finally, how did we do with the data analysis? We did well: over 70% accuracy in predicting the close of the S&P 500 is the highest we've seen achieved on this dataset, so with few steps and a few lines of code we've produced a full-on machine learning model. The reason for the relatively modest accuracy achieved is the dataset itself; there isn't enough signal there to do significantly better. But 7 times out of 10, we were able to correctly determine if the S&P 500 index would close up or down on the day, and that's objectively good.

## Cleanup

**When you're finished, shut down the managed VM you used for Cloud Datalab to avoid incurring costs.**